In [3]:
import numpy as np
import pandas as pd
from qiskit.quantum_info import SparsePauliOp

In [4]:
def create_matrix(cut_off, type):
    # Initialize a zero matrix of the specified size
    matrix = np.zeros((cut_off, cut_off), dtype=np.complex128)
    
    # Fill the off-diagonal values with square roots of integers
    for i in range(cut_off):
        if i > 0:  # Fill left off-diagonal
            if type == 'q':
                matrix[i][i - 1] = (1/np.sqrt(2)) * np.sqrt(i)  # sqrt(i) for left off-diagonal
            else:
                matrix[i][i - 1] = (1j/np.sqrt(2)) * np.sqrt(i)

        if i < cut_off - 1:  # Fill right off-diagonal
            if type == 'q':
                matrix[i][i + 1] = (1/np.sqrt(2)) * np.sqrt(i + 1)  # sqrt(i + 1) for right off-diagonal
            else:
                matrix[i][i + 1] = (-1j/np.sqrt(2)) * np.sqrt(i + 1)

    return matrix


In [5]:
# Function to calculate the Hamiltonian
def calculate_Hamiltonian(cut_off, potential):
    # Generate the position (q) and momentum (p) matrices
    q = create_matrix(cut_off, 'q')  # q matrix
    p = create_matrix(cut_off, 'p')  # p matrix

    # Calculate q^2 and q^3 for potential terms
    q2 = np.matmul(q, q)
    q3 = np.matmul(q2, q)

    #fermionic identity
    I_f = np.eye(2)

    #bosonic identity
    I_b = np.eye(cut_off)

    # Superpotential derivatives
    if potential == 'QHO':
        W_prime = q  # W'(q) = q
        W_double_prime = I_b #W''(q) = 1

    elif potential == 'AHO':
        W_prime = q + q3  # W'(q) = q + q^3
        W_double_prime = I_b + 3 * q2  # W''(q) = 1 + 3q^2

    elif potential == 'DW':
        W_prime = q + q2 + I_b  # W'(q) = q + q^2 + 1
        W_double_prime = I_b + 2 * q  # W''(q) = 1 + 2q

    else:
        print("Not a valid potential")
        raise

    # Kinetic term: p^2
    p2 = np.matmul(p, p)

    # Commutator term [b^†, b] = -Z
    Z = np.array([[1, 0], [0, -1]])  # Pauli Z matrix for fermion number
    commutator_term = np.kron(Z, W_double_prime)

    # Construct the block-diagonal kinetic term (bosonic and fermionic parts)
    # Bosonic part is the same for both, hence we use kron with the identity matrix
    kinetic_term = np.kron(I_f, p2)

    # Potential term (W' contribution)
    potential_term = np.kron(I_f, np.matmul(W_prime, W_prime))

    # Construct the full Hamiltonian
    H_SQM = 0.5 * (kinetic_term + potential_term + commutator_term)
    H_SQM[np.abs(H_SQM) < 10e-12] = 0
    
    return H_SQM

In [43]:
data = {'cut_off':[],
       'QHO': [],
       'AHO': [],
       'DW': []
       }

num_cut_offs = 10
cut_off_list = [2**i for i in range(1, num_cut_offs + 1)]
potential_list = ['QHO', 'AHO', 'DW']

for cut_off in cut_off_list:
    data['cut_off'].append(cut_off)

    for potential in potential_list:

        H = calculate_Hamiltonian(cut_off, potential)
        H_pauli = SparsePauliOp.from_operator(H)

        data[potential].append(H_pauli.size)

In [44]:
data

{'cut_off': [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024],
 'QHO': [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024],
 'AHO': [2, 10, 34, 102, 270, 670, 1548, 3496, 7486, 15534],
 'DW': [4, 14, 48, 136, 352, 854, 1990, 4450, 9874, 21202]}

In [45]:
pauli_df = pd.DataFrame(data).set_index('cut_off')

In [46]:
pauli_df

,QHO,AHO,DW
cut_off,,,
2,2,2,4
4,4,10,14
8,8,34,48
16,16,102,136
32,32,270,352
64,64,670,854
128,128,1548,1990
256,256,3496,4450
512,512,7486,9874


In [74]:
H = calculate_Hamiltonian(4, 'AHO')
H_pauli = SparsePauliOp.from_operator(H)
#H2 = H_pauli.simplify(atol=1e-4)

In [82]:
H_pauli.size

10

In [76]:
from qiskit.quantum_info import Z2Symmetries

In [77]:
symmetries = Z2Symmetries.find_z2_symmetries(H_pauli)
tapered_hamiltonian = symmetries.taper(H_pauli)

In [78]:
H = sum(tapered_hamiltonian)

In [83]:
H.size

10

In [79]:
np.sort(np.linalg.eig(H)[0])

array([ 9.28279701+0.j, 73.21720299+0.j])

In [33]:
len(H2)

32

In [25]:
len(H2)

1

In [ ]:
data = {'cut_off':[],
       'QHO': [],
       'AHO': [],
       'DW': []
       }

num_cut_offs = 10
cut_off_list = [2**i for i in range(1, num_cut_offs + 1)]
potential_list = ['QHO', 'AHO', 'DW']

for cut_off in cut_off_list:
    data['cut_off'].append(cut_off)

    for potential in potential_list:

        H = calculate_Hamiltonian(cut_off, potential)
        H_pauli = SparsePauliOp.from_operator(H)

        data[potential].append({'hamiltonian_size': H_pauli.dim,
                        'num_paulis': len(H_pauli),
                        'num_qubits': H_pauli.num_qubits})

